In [1]:
import os, sys
import pybedtools as pbt

import config_readwrite as crw

In [2]:
cfn_file = os.path.join(os.getcwd(), "config.bat2.ini")

config, cfn = crw.read(cfn_file)

## functions

In [4]:
def liftOver(bed_file, to_build, from_build, minmatch, chainfile):

    src = "/wynton/group/ahituv/fongsl/tools/evo/liftover_bed-wynton.py"
    cmd = " ".join([
        "python",
        src,
        bed_file,
        from_build,
        to_build,
        "-m",
        minmatch,
        "-c",
        chainfile

    ])

    print(cmd)

    # run command in command line
    os.system(cmd)

## params

In [6]:
tissue='liver'
species = "ajam"

sections = [
    #('efus', 'eptfus_hg38', 'hg38_eptfus'),
    (tissue, 'artjam_hg38', 'hg38_artjam'),         
           ]

species_specific=['ajam_specific']

liftover_runs = {
    "ajam":('hg38', "artJam2"), 
    "efus":('hg38', "eptFus1"),   
}

formal_names ={
    "ajam":("Artibeus_jamaicensis","fruit_a500"), 
    "efus":('Eptesicus_fuscus', "insect_a500")
}

BUILD = "hg38"
MINMATCH = "0.1"

## liftover hg38 bed -> ArtJam2 bed

In [8]:
for tissue, chain1, chain2 in sections:
    for sp in species_specific:
        
        BED = config[tissue][sp]
        CHAIN = config["chain"][chain2]  # hu -> bat
        
        # get naming conventions
        formal_name, conv_name = formal_names[species]
        PATH = os.path.dirname(BED)
        print(BED, CHAIN, formal_name, conv_name, PATH)
        

         # write to config
        config[tissue][f"{tissue}.{species}_specific.{chain2}.bed"]= BED
        
         # get the to and from builds, 
        from_build, to_build = liftover_runs[species]
        print(from_build, to_build)

        # do liftover
        liftOver(BED, to_build, from_build, MINMATCH, CHAIN)
        
        # get liftover filename
        LIFTOVER_FILE = os.path.join(PATH, f"{tissue}.{species}.specific.liftOver.to.{to_build}.bed")
        print(LIFTOVER_FILE)
        
        # write liftover to config
        config[tissue][f"{sp}.{from_build}.to.{to_build}"] = LIFTOVER_FILE
        

# write new beds to config
crw.write(config, cfn)

/wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver/liver.ajam.specific.bed /wynton/home/ahituv/fongsl/other_analyses/for-wei_bats/data/chain/hg38.chr1_22.ArtJamW.chain.final.gz Artibeus_jamaicensis fruit_a500 /wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver
hg38 artJam2
python /wynton/group/ahituv/fongsl/tools/evo/liftover_bed-wynton.py /wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver/liver.ajam.specific.bed hg38 artJam2 -m 0.1 -c /wynton/home/ahituv/fongsl/other_analyses/for-wei_bats/data/chain/hg38.chr1_22.ArtJamW.chain.final.gz


Reading liftover chains
Mapping coordinates


/wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver 0.1 <class 'str'>


lifting over /wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver/liver.ajam.specific.bed from hg38 to artJam2 in /wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver 


Sorting .bed /wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver/temp_liver.ajam.specific.bed
found empty file

liftingOver liver.ajam.specific 

 /wynton/group/ahituv/bin/liftOver /wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver/temp_liver.ajam.specific.bed /wynton/home/ahituv/fongsl/other_analyses/for-wei_bats/data/chain/hg38.chr1_22.ArtJamW.chain.final.gz /wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for

## ArtJam2 Bed -> ArtJam2 FASTA sequences. 

In [11]:
LIFTED_BED = "/wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver/liver.ajam.specific.liftOver.to.artJam2.bed"
FASTA="/wynton/group/ahituv/data/dna/artjam2/ArtJam2_scaffolds_largerthan_50kb.fa"
OUT = LIFTED_BED.strip(".bed") + ".fa"
cmd = ["bedtools getfasta", 
      "-fi", 
      FASTA, 
      "-bed", 
      LIFTED_BED,
      ">", 
       OUT
      ]

print(" ".join(cmd))

bedtools getfasta -fi /wynton/group/ahituv/data/dna/artjam2/ArtJam2_scaffolds_largerthan_50kb.fa -bed /wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver/liver.ajam.specific.liftOver.to.artJam2.bed > /wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver/liver.ajam.specific.liftOver.to.artJam2.fa


## write fasta file to config

In [15]:
config[tissue][f"{tissue}.{species}_specific.{chain2}.fa"] = OUT
# write new beds to config
crw.write(config, cfn)

In [13]:
OUT

'/wynton/home/ahituv/fongsl/other_analyses/for-hai_bats/data/CallPeaks_results_for_Sarah/Artibeus_jamaicensis/liver/liver.ajam.specific.liftOver.to.artJam2.fa'